In [66]:
import numpy as np
import pickle
import pandas as pd
import glob
import os
from nilearn.connectome import ConnectivityMeasure
import matplotlib.pyplot as plt 
from nilearn import plotting

import seaborn as sns

/home/hannahcraddock/anaconda3/envs/hannah_env/lib/python3.9/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [64]:
import pandas as pd

In [2]:
def get_rsfmri_adj_matrix(root_pth):
    '''Get resting state & return adjacency matrix '''
    
    #Connectivity
    correlation_measure = ConnectivityMeasure(kind='correlation')
    #Resting state fmri 
    rest_task = 'Rest'
    corr_rs_fmri = []

    #Resting state files 
    for f in glob.glob(os.path.join(root_pth, rest_task)+'/*.txt'):
        subj = f.split('/')[-1].split('_')[0]
        filename = os.path.join(root_pth, rest_task) +'/'+ subj +'_schaefer_400ROI_'+ rest_task.lower() +'.txt'
        #print(filename)
        ts_df = np.loadtxt(filename)
        correlation_matrix = correlation_measure.fit_transform([ts_df])[0]
        corr_rs_fmri.append(correlation_matrix)

    adj_matrix = np.array(corr_rs_fmri).mean(axis=0) #Is it the first access - yes if size 644 x... 

    return adj_matrix

In [3]:
def plot_corr_matrix(correlation_matrix):

    fig, ax = plt.subplots()
    #ax.plot([1,2,3,4,5])
    #plt.figure(1)
    x = np.linspace(0, 20, 100)
    ax.plot(x, np.sin(x))
    plt.show(block=False)

    #hmap = sns.heatmap(correlation_matrix)

In [3]:
def get_fmri_data(root_pth, task_type):
    '''Get fmri data for a given task ''' 
    fmri_total = []
    subj_list = []
    for f in glob.glob(os.path.join(root_pth, task_type)+'/*.txt'):
        subj = f.split('/')[-1].split('_')[0]
        subj_list.append(subj)
        filename = os.path.join(root_pth, task_type)+'/'+ subj +'_schaefer_400ROI_'+ task_type.lower()+'.txt'
        #print(filename)
        ts_df = np.loadtxt(filename)
        fmri_total.append(ts_df)
    
    return fmri_total, subj_list

#### Apply Function

In [ ]:
root_pth = '/camcan/schaefer_parc/'
#Adjacency matrix 
adj_mat = get_rsfmri_adj_matrix(root_pth)

Error: Pip module Unable to parse debugpy output, please log an issue with https://github.com/microsoft/vscode-jupyter is required for debugging cells. You will need to install it to debug cells.

#### fmri

In [5]:
root_pth = '/camcan/schaefer_parc/'
fmri_movie, subj_list = get_fmri_data(root_pth, 'Movie')
print(np.array(fmri_movie).shape) #(193, 400)

(644, 193, 400)


In [8]:
np.max(fmri_movie)

1136.735294

In [9]:
np.min(fmri_movie)

35.677419

In [10]:
#Dictionary

dictX = {'vis': {0: [1,11,1300], 1: [3,9,1400], 2: [2,12,1345]},
          'dorsal': {0: [172,9,1], 1: [120,14,1], 2: [100,12,1]}}

In [88]:
dictX

{'vis': {0: [1, 11, 1300], 1: [3, 9, 1400], 2: [2, 12, 1345]},
 'dorsal': {0: [172, 9, 1], 1: [120, 14, 1], 2: [100, 12, 1]}}

In [89]:
liX = list(iter(dictX.values()))
liX

[{0: [1, 11, 1300], 1: [3, 9, 1400], 2: [2, 12, 1345]},
 {0: [172, 9, 1], 1: [120, 14, 1], 2: [100, 12, 1]}]

dictX.keys()

In [ ]:
def get_stats_dict ():

    dict_results = {}
    d_temp = {}
    df_results = pd.DataFrame()
    list_net = []; list_buckets = []; list_mean = []; list_std = []
    for net in dictX.keys():
        print(net)
        l = list(iter(dictX[net].values()))
        dict_results[net] = 0
        for i in range(len(l[0])):
            print(i)
            list_net.append(net)
            list_buckets.append(i)
            row_list = [row[i] for row in l] #get values column-wise
            print(row_list)
            list_mean.append(np.mean(np.array(row_list)))
            list_std.append(np.std(np.array(row_list)))
    #Dataframe
    df_results['network'] = list_net
    df_results['time_bucket'] = list_buckets
    df_results['mean_pcent_fcn'] = list_mean
    df_results['std_pcent_fcn'] = list_std

In [69]:
dict_results = {}
d_temp = {}
df_results = pd.DataFrame()
list_net = []; list_buckets = []; list_mean = []; list_std = []
for net in dictX.keys():
    print(net)
    l = list(iter(dictX[net].values()))
    dict_results[net] = 0
    for i in range(len(l[0])):
        print(i)
        list_net.append(net)
        list_buckets.append(i)
        row_list = [row[i] for row in l] #get values column-wise
        print(row_list)
        list_mean.append(np.mean(np.array(row_list)))
        list_std.append(np.std(np.array(row_list)))
#Dataframe
df_results['network'] = list_net
df_results['time_bucket'] = list_buckets
df_results['mean_pcent_fcn'] = list_mean
df_results['std_pcent_fcn'] = list_std

vis
0
[1, 3, 2]
1
[11, 9, 12]
2
[1300, 1400, 1345]
dorsal
0
[172, 120, 100]
1
[9, 14, 12]
2
[1, 1, 1]


In [68]:
df_results

,network,time_bucket,mean_pcent_fcn,std_pcent_fcn
0,vis,0,2.000000,0.816497
1,vis,1,10.666667,1.247219
2,vis,2,1348.333333,40.892814
3,dorsal,0,130.666667,30.346151
4,dorsal,1,11.666667,2.054805
5,dorsal,2,1.000000,0.000000


In [ ]:
#### Check results

In [84]:
def get_network_pcent(dict_netw_results, n_repeat):
    list_net = ['vis', 'dor', 'motor']
    for networkX in list_net:
        print(f'Network = {networkX}')
        
        #Append percentage correct for each network
        dict_netw_results[networkX] = {n_repeat: 100*np.array([1,2,3,4])}

        return dict_netw_results

In [86]:
dict_netw_results = {}
list_net = []; list_buckets = []; list_mean = [];
for i in np.arange(2, 10):
    #list_net.append()
    list_buckets.append(i)
    print(i)
    dict_netw_results = get_network_pcent(dict_netw_results, i)

2
Network = vis
3
Network = vis
4
Network = vis
5
Network = vis
6
Network = vis
7
Network = vis
8
Network = vis
9
Network = vis


In [ ]:
dict_netw_results = {}
list_net = []; list_buckets = []; list_mean = [];
list_net = ['vis', 'dor', 'motor']
dict_netw_results = {key: {} for key in list_net}

#dict_netw_results.keys = list_net
#dict_netw_results.fromkeys(list_net, None)
print(dict_netw_results)
for i in np.arange(1, 10):
    #list_net.append()
    list_buckets.append(i)
    print(i)
    #dict_netw_results = get_network_pcent(dict_netw_results, i)
    for networkX in list_net:
        print(f'Network = {networkX}')
        
        #Append percentage correct for each network
        dict_netw_results[networkX][i] = 100*np.array([1,2,3,4])


In [ ]:
dict_netw_results = {key: {} for key in self.list_networks}

#Repeat to get std
for i in np.arange(1, self.n_model_repeats):
    print('Iteration num: {}'.format(i))
    #Repeat for each network
    for networkX in self.list_networks:
        print(f'Network = {networkX}')
        fmri_networkX, n_regions = self.get_network_fmri(networkX)  # fmri or self.fmri??
        best_acc, best_prop = self.run_model(
            fmri_networkX, n_regions)
        dict_netw_results[networkX][i] = 100*(best_prop/self.num_test_sub)
    
    df_stats_results = get_stats_network_results(dict_netw_results)

In [96]:
dict_netw_results

{'vis': {2: array([100, 200, 300, 400]),
  3: array([100, 200, 300, 400]),
  4: array([100, 200, 300, 400]),
  5: array([100, 200, 300, 400]),
  6: array([100, 200, 300, 400]),
  7: array([100, 200, 300, 400]),
  8: array([100, 200, 300, 400]),
  9: array([100, 200, 300, 400])},
 'dor': {2: array([100, 200, 300, 400]),
  3: array([100, 200, 300, 400]),
  4: array([100, 200, 300, 400]),
  5: array([100, 200, 300, 400]),
  6: array([100, 200, 300, 400]),
  7: array([100, 200, 300, 400]),
  8: array([100, 200, 300, 400]),
  9: array([100, 200, 300, 400])},
 'motor': {2: array([100, 200, 300, 400]),
  3: array([100, 200, 300, 400]),
  4: array([100, 200, 300, 400]),
  5: array([100, 200, 300, 400]),
  6: array([100, 200, 300, 400]),
  7: array([100, 200, 300, 400]),
  8: array([100, 200, 300, 400]),
  9: array([100, 200, 300, 400])}}

In [102]:

def get_stats_network_results(dict_netw_results):
    df_results = pd.DataFrame()
    list_net = []; list_buckets = []; list_mean = []; list_std = []
    for net in dict_netw_results.keys():
        #print(net)
        l = list(iter(dict_netw_results[net].values()))
        for i in range(len(l[0])):
            #print(i)
            list_net.append(net)
            list_buckets.append(i)
            row_list = [row[i] for row in l] #get values column-wise
            #print(row_list)
            list_mean.append(np.mean(np.array(row_list)))
            list_std.append(np.std(np.array(row_list)))
    #Dataframe
    df_results['network'] = list_net
    df_results['time_bucket'] = list_buckets
    df_results['mean_pcent_fcn'] = list_mean
    df_results['std_pcent_fcn'] = list_std

    return df_results

In [103]:
df_results = get_stats_network_results(dict_netw_results)
df_results

,network,time_bucket,mean_pcent_fcn,std_pcent_fcn
0,vis,0,100.0,0.0
1,vis,1,200.0,0.0
2,vis,2,300.0,0.0
3,vis,3,400.0,0.0
4,dor,0,100.0,0.0
5,dor,1,200.0,0.0
6,dor,2,300.0,0.0
7,dor,3,400.0,0.0
8,motor,0,100.0,0.0
9,motor,1,200.0,0.0


In [99]:
df_results

,network,time_bucket,mean_pcent_fcn,std_pcent_fcn
0,vis,0,100.0,0.0
1,vis,1,200.0,0.0
2,vis,2,300.0,0.0
3,vis,3,400.0,0.0
4,dor,0,100.0,0.0
5,dor,1,200.0,0.0
6,dor,2,300.0,0.0
7,dor,3,400.0,0.0
8,motor,0,100.0,0.0
9,motor,1,200.0,0.0


In [62]:
dict_results = {}
d_temp = {}
for net in dictX.keys():
    print(net)
    l = list(iter(dictX[net].values()))
    print(len(l[0]))
    #print(l)
    dict_results[net] = 0
    for i in range(len(l[0])):
        print(i)
        row_list = [row[i] for row in l] #get values column-wise
        print(row_list)
        #dict2[net] = {i: sum(row_list)/len(row_list)} #{i:sum(row_list)/len(row_list)}
        d_temp[net] = {i: sum(row_list)/len(row_list)}
        dict_results[net].update(d_temp[net])
        #d[i] = sum(row_list)/len(row_list) #calculate avg
     

vis
3
0
[1, 3, 2]


AttributeError: 'int' object has no attribute 'update'

In [59]:
dict_results

{'vis': {2: 1348.3333333333333}, 'dorsal': {2: 1.0}}

In [ ]:
#The difference;
[[20, 50, 70, 90, 100], [10, 3, 0, 99, 57], [90, 1000, 2, 3.4, 5]]

In [21]:
l = list(iter(dictX.values()))                      #convert values to list

d={} 
#print(len(l[0]))                                                                 #final ditionary
for i in range(len(l[0])):
    print(i)
    row_list = [row[i] for row in l]                     #get values column-wise
    print(sum(row_list))
    d[i] = sum(row_list)/len(row_list)               #calculate avg


3
0


TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [ ]:
for key, group in df.groupby('quality'):
    group.plot('insert', 'mean', yerr='std', label=key, ax=ax) 